In [1]:
import datetime
import gpm_api
import pyvista as pv
from gpm_api.visualization.plot_3d import (
    create_pyvista_2d_surface, 
    create_pyvista_3d_volume,
)

# Load GPM DPR overpass

In [2]:
# Specify the time period you are interested in 
start_time = datetime.datetime.strptime("2020-07-05 02:00:00", "%Y-%m-%d %H:%M:%S")
end_time = datetime.datetime.strptime("2020-07-05 06:00:00", "%Y-%m-%d %H:%M:%S")
# Specify the product and product type 
product = "2A-DPR"    # 2A-PR
product_type = "RS"   
# Specify the version
version = 7

# Load the 2A-DPR dataset
# - If scan_mode is not specified, it automatically load one! 
ds = gpm_api.open_dataset(
    product=product,
    product_type=product_type,
    version=version,
    start_time=start_time,
    end_time=end_time,
)
ds

isel_dict = {"cross_track": slice(0,49), "along_track": slice(884, 984)}
ds_patch = ds.isel(isel_dict)

'scan_mode' has not been specified. Default to FS.


# Load GPM DPR overpass of Hurricane Zeta

In [3]:
# Define analysis time period
start_time = datetime.datetime.strptime("2020-10-28 08:00:00", "%Y-%m-%d %H:%M:%S")
end_time = datetime.datetime.strptime("2020-10-28 09:00:00", "%Y-%m-%d %H:%M:%S")

# Define products to analyze
product = "2A-DPR"    # 2A-PR
product_type = "RS"  
version = 7

# Load the 2A-DPR dataset
# - If scan_mode is not specified, it automatically load one! 
ds = gpm_api.open_dataset(
    product=product,
    product_type=product_type,
    version=version,
    start_time=start_time,
    end_time=end_time,
)
ds

bbox = [-94, -89, 22.5, 27.5]
ds_patch = ds.gpm_api.crop(bbox)

'scan_mode' has not been specified. Default to FS.


# Create PyVista objects

In [4]:
# Retrieve pyvista surface and volume data objects
radar_frequency="Ku"
da_2d = ds_patch["zFactorFinalNearSurface"].sel(radar_frequency=radar_frequency).compute()
surf = create_pyvista_2d_surface(da_2d, spacing=(1, 1, 1), origin=(0, 0, -3))

da_3d = ds_patch["zFactorFinal"].sel(radar_frequency=radar_frequency).compute()
vol = create_pyvista_3d_volume(da_3d, spacing=(1, 1, 0.25), origin=(0, 0, 0))

In [5]:
# Define figure settings 
cmap="Spectral_r"
clim=[10, 50]

# 2D Surface Map Visualization

In [6]:
# Display
pl = pv.Plotter()
pl.background_color = "#2e2a2b"
pl.add_mesh(surf, opacity=1, cmap=cmap, clim=clim, show_scalar_bar=False)
pl.show()

Widget(value="<iframe src='http://localhost:45545/index.html?ui=P_0x7fd8d2dda3d0_0&reconnect=auto' style='widt…

# 3D Isosurfaces Visualization

In [7]:
from gpm_api.visualization.plot_3d import add_3d_isosurfaces

isovalues=[30, 40, 50]
opacities=[0.3, 0.5, 1]
method="contour"

# Display figure 
pl = pv.Plotter([1024*2, 1024*3], image_scale=3)
pl.background_color = "#2e2a2b"
pl.add_mesh(surf, opacity=1, cmap=cmap, clim=clim, show_scalar_bar=False, style="surface")
add_3d_isosurfaces(vol=vol, pl=pl, 
                   isovalues=isovalues,
                   opacities=opacities,
                   method=method,
                   cmap=cmap,
                   clim=clim,
                   show_scalar_bar=False,
                   add_sliders=True,
                  )
pl.show()

Widget(value="<iframe src='http://localhost:45545/index.html?ui=P_0x7fd8d2d5bf90_1&reconnect=auto' style='widt…

In [8]:
# Display figure 
pl = pv.Plotter([1024*2, 1024*3], image_scale=3)
pl.background_color = "#2e2a2b"
pl.add_mesh(surf, opacity=1, cmap=cmap, clim=clim, show_scalar_bar=False, style="surface")
add_3d_isosurfaces(vol=vol, pl=pl, 
                   isovalues=isovalues,
                   opacities=opacities,
                   method=method,
                   cmap=cmap,
                   clim=clim,
                   show_scalar_bar=False,
                   add_sliders=False,
                  )
pl.show()

Widget(value="<iframe src='http://localhost:45545/index.html?ui=P_0x7fd8d34b0210_2&reconnect=auto' style='widt…

# 3D Isosurface Interactive Slider

In [9]:
import pyvista as pv
from gpm_api.visualization.plot_3d import add_3d_isosurface_slider

# Display figure 
pl = pv.Plotter([1024*2, 1024*3], image_scale=3)
pl.background_color = "#2e2a2b"
pl.add_mesh(surf, opacity=1, cmap=cmap, clim=clim, show_scalar_bar=False, style="surface")
add_3d_isosurface_slider(vol=vol, pl=pl, isovalue=30, opacity=1, cmap=cmap, clim=clim, show_scalar_bar=False, style="surface")
pl.show()

Widget(value="<iframe src='http://localhost:45545/index.html?ui=P_0x7fd8d018ac50_3&reconnect=auto' style='widt…

# 3D Orthogonal Slices Visualization

In [10]:
from gpm_api.visualization.plot_3d import add_3d_orthogonal_slices

pl = pv.Plotter([1024*3, 768*3], image_scale=3)
pl.background_color = "#2e2a2b" 
add_3d_orthogonal_slices(vol=vol, pl=pl, x=22, y=20, z=1, add_sliders=True, clim=clim, cmap=cmap, show_scalar_bar=False)
pl.add_scalar_bar(vertical=True, title="Reflectivity [dBZ]", height=0.4, position_x=0.85)
pl.show()

Widget(value="<iframe src='http://localhost:45545/index.html?ui=P_0x7fd8d2331190_4&reconnect=auto' style='widt…

# 3D Volume Single Plane Slicer 

In [11]:
pl = pv.Plotter()
pl.background_color = "#2e2a2b"
pl.add_mesh(surf, clim=clim, cmap=cmap, show_scalar_bar=False)
pl.add_volume_clip_plane(vol, normal='-x', cmap=cmap, clim=clim)
pl.show()

Widget(value="<iframe src='http://localhost:45545/index.html?ui=P_0x7fd8d0171850_5&reconnect=auto' style='widt…

# 3D Volume Vertical Slicer

In [12]:
pl = pv.Plotter()
pl.background_color = "#2e2a2b"
pl.add_volume_clip_plane(vol, normal='-z', normal_rotation=False, cmap=cmap, clim=clim)
pl.show()

Widget(value="<iframe src='http://localhost:45545/index.html?ui=P_0x7fd8d01f4690_6&reconnect=auto' style='widt…

# 3D Volume Orthogonal Slicer

In [13]:
pl = pv.Plotter()
pl.background_color = "#2e2a2b"
pl.add_mesh(surf, clim=clim, cmap=cmap, show_scalar_bar=False)
pl.add_volume_clip_plane(
    vol,
    normal='-x',
    interaction_event='always',
    normal_rotation=False,
    cmap=cmap, clim=clim,
    show_scalar_bar=False,
)
pl.add_volume_clip_plane(
    vol,
    normal='-y',
    interaction_event='always',
    normal_rotation=False,
    cmap=cmap, clim=clim,
    show_scalar_bar=False,
)
pl.show()

Widget(value="<iframe src='http://localhost:45545/index.html?ui=P_0x7fd8d0243750_7&reconnect=auto' style='widt…